In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
import joblib
from tqdm import tqdm
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from preprocessing.noise_reduction import NoiseReducer
from preprocessing.silence_removal import SilenceRemover
from preprocessing.speech_filter import SpeechFilter
from preprocessing.identity import Identity


from audio import Audio

original_metadata_path = os.path.join(".", "data", "original_data_labeled.tsv")
filtered_metadata_path = os.path.join(".", "data", "filtered_data_labeled.tsv")
audio_dir = os.path.join(".", "data", "filtered_clips")

### Pipeline

In [96]:

folder_path_MFCC = 'trials/features/Mfcc_extra_stats_2000_n'
folder_path_HFCC = 'trials/features/Hfcc_extra_stats_2000_n'

X_loaded_MFCC = joblib.load(os.path.join(folder_path_MFCC, 'X.joblib'))
y_loaded_MFCC = joblib.load(os.path.join(folder_path_MFCC, 'y.joblib'))

X_loaded_HFCC = joblib.load(os.path.join(folder_path_HFCC, 'X.joblib'))
y_loaded_HFCC = joblib.load(os.path.join(folder_path_HFCC, 'y.joblib'))

print(X_loaded_MFCC.shape)
print(y_loaded_MFCC.shape)
print(X_loaded_HFCC.shape)
print(y_loaded_HFCC.shape)
# MFCC mfcc75mean, mfcc75std, feature_min, feature_max
# HFCC mfcc75mean, mfcc75std, feature75median, feature_q25, feature_q75
X_loaded_MFCC_modified = np.delete(X_loaded_MFCC, np.s_[150:450], axis=1)
X_loaded_HFCC_modified = X_loaded_HFCC[:,:375]
X_concatenated = np.concatenate((X_loaded_MFCC_modified, X_loaded_HFCC_modified), axis=1)

print(X_concatenated.shape)


(8000, 600)
(8000,)
(8000, 525)
(8000,)
(8000, 675)


In [3]:
def load_and_concatenate_features(features, main_folder='extracted_features'):
    """
    Load and concatenate features from a single folder.
    
    Args:
        folders (list): List of feature names (used for validation)
        main_folder (str): Main directory containing all features
        
    Returns:
        tuple: (X_train_combined, X_test_combined, y_train, y_test)
    """
    X_train_arrays = []
    X_test_arrays = []
    
    # Load y labels only once since they're the same for all features
    y_train = joblib.load(os.path.join(main_folder, 'y_train.joblib'))
    y_test = joblib.load(os.path.join(main_folder, 'y_test.joblib'))
    
    for feature in features:
        # Construct file names for train and test data
        train_file = f"{feature}_train.joblib"
        test_file = f"{feature}_test.joblib"
        
        # Validate existence of files
        train_path = os.path.join(main_folder, train_file)
        test_path = os.path.join(main_folder, test_file)
        
        if not os.path.exists(train_path) or not os.path.exists(test_path):
            raise ValueError(f"Missing train or test file for feature: {feature}")
        
        # Load train and test data
        X_train = joblib.load(train_path)
        X_test = joblib.load(test_path)
        
        if X_train.shape == np.ravel(X_train).shape:
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)
            print(f"Reshaped X_train for {feature}: {X_train.shape}")
        X_train_arrays.append(X_train)
        X_test_arrays.append(X_test)
    
    # Concatenate along feature axis (axis=1)
    X_train_combined = np.concatenate(X_train_arrays, axis=1)
    X_test_combined = np.concatenate(X_test_arrays, axis=1)
    
    return X_train_combined, X_test_combined, y_train, y_test

In [4]:
x_train_combined, x_test_combined, y_train, y_test = load_and_concatenate_features(['mfcc150','hfcc150','cpps','fO_1','jitter','pitch_range_1','pitch_mean_min_max_freq_3','alpha_ratio1','spectral5'],
main_folder='trials/features/all_classes_7_ft_splitted')


print(f"x_train_combined shape: {x_train_combined.shape}")
print(f"x_test_combined shape: {x_test_combined.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Reshaped X_train for alpha_ratio1: (145504, 1)
x_train_combined shape: (145504, 313)
x_test_combined shape: (36376, 313)
y_train shape: (145504,)
y_test shape: (36376,)


In [99]:
X_train_loaded = X_concatenated
y_train_loaded = y_loaded_MFCC % 2
# y_train_loaded = y_loaded 
# y_test_loaded = y_test % 2
print(X_train_loaded.shape, y_train_loaded.shape)

(8000, 675) (8000,)


In [100]:

def display_highest_accuracy(highest_accuracy, best_combination):
    """
    Display the highest accuracy and its corresponding feature combination in a styled Markdown block.

    Args:
        highest_accuracy (float): The highest accuracy achieved.
        best_combination (list): The best feature combination.
    """
    display(Markdown(f"""
<div style="border: 2px solid #424242; padding: 15px; border-radius: 12px; background-color: #212121; color: #E0E0E0;">
    <h2 style="color: #FFCA28; text-align: center; font-family: Arial, sans-serif;">🌟 Current Highest Accuracy 🌟</h2>
    <p style="font-size: 18px; text-align: center; font-family: Arial, sans-serif;">
        <strong>Accuracy:</strong> <span style="color: #76FF03; font-weight: bold;">{highest_accuracy:.6f}</span><br>
        <strong>Combination:</strong> <span style="color: #29B6F6; font-weight: bold;">{(best_combination)}</span>
    </p>
</div>
"""))



In [104]:
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from IPython.display import display, Markdown


def find_best_feature_combination(X, y, feature_indices):
    """
    Find the best feature combination for classification, including non-consecutive combinations, and store all accuracies.

    Args:
        X (numpy.ndarray): Feature vectors.
        y (numpy.ndarray): Labels.
        feature_indices (list of tuples): List of (start, end) indices for each feature vector.

    Returns:
        dict: Best combination, accuracy, feature indices, and all accuracies.
    """
    best_accuracy = 0
    best_combination = None
    best_features = None
    all_accuracies = []

    # Generate all possible feature combinations
    print("Generating all possible feature combinations...")
    print(f"Feature indices: {feature_indices}")
    total_combinations = sum(len(list(combinations(feature_indices, r))) for r in range(1, len(feature_indices) + 1))
    print(f"Total combinations to evaluate: {total_combinations}")
    with tqdm(total=total_combinations, desc="Evaluating feature combinations") as pbar:
        for r in range(1, len(feature_indices) + 1):
            for combination in combinations(feature_indices, r):
                # Extract features for the current combination
                selected_features = []
                for start, end in combination:
                    selected_features.append(X[:, start:end])
                X_combined = np.hstack(selected_features)

                # Perform cross-validation
                clf = SVC(kernel='rbf', C=1000, gamma=0.0001, random_state=42, class_weight='balanced')
                #clf = knn(n_neighbors=4, weights='distance', algorithm='brute', n_jobs=-1)
                X_train, X_val, y_train, y_val = train_test_split(X_combined, y, test_size=0.2, random_state=42)
                clf.fit(X_train, y_train)
                cv_scores = [accuracy_score(y_val, clf.predict(X_val))]

                # Calculate the mean cross-validation score
                mean_cv_score = np.mean(cv_scores)

                # Store the mean cross-validation score and combination
                all_accuracies.append({"combination": combination, "accuracy": mean_cv_score})

                # Update the best combination if current accuracy is higher
                if mean_cv_score > best_accuracy:
                    best_accuracy = mean_cv_score
                    best_combination = combination
                    best_features = X_combined

                
                pbar.update(1)

    return {
        "best_combination": best_combination,
        "best_accuracy": best_accuracy,
        "best_features": best_features,
        "all_accuracies": all_accuracies
    }

feature_indices = [(0, 75), (75,150),(150, 225),(225, 300),(300, 375),(375, 450),(450, 525),(525, 600),(600, 675)]
mapping = {
    "mfcc75mean": (0, 75),
    "mfcc75std": (75, 150),
    "MFCC_min": (150, 225),
    "MFCC_max": (225, 300),
    "Hfcc75mean" : (300, 375),
    "Hfcc75std" : (375, 450),
    "HFCC75median" : (450, 525),
    "HFCC_q25" : (525, 600),
    "HFCC_q75" : (600, 675),
}

result = find_best_feature_combination(X_train_loaded, y_train_loaded, feature_indices)
print("Best Combination:", result["best_combination"])
print("Best Accuracy:", result["best_accuracy"])
print("All Accuracies:", result["all_accuracies"])


Generating all possible feature combinations...
Feature indices: [(0, 75), (75, 150), (150, 225), (225, 300), (300, 375), (375, 450), (450, 525), (525, 600), (600, 675)]
Total combinations to evaluate: 511


Evaluating feature combinations: 100%|██████████| 511/511 [56:03<00:00,  6.58s/it]

Best Combination: ((0, 75), (150, 225), (225, 300))
Best Accuracy: 0.97125
All Accuracies: [{'combination': ((0, 75),), 'accuracy': 0.921875}, {'combination': ((75, 150),), 'accuracy': 0.946875}, {'combination': ((150, 225),), 'accuracy': 0.85125}, {'combination': ((225, 300),), 'accuracy': 0.946875}, {'combination': ((300, 375),), 'accuracy': 0.90875}, {'combination': ((375, 450),), 'accuracy': 0.9275}, {'combination': ((450, 525),), 'accuracy': 0.875625}, {'combination': ((525, 600),), 'accuracy': 0.74}, {'combination': ((600, 675),), 'accuracy': 0.93875}, {'combination': ((0, 75), (75, 150)), 'accuracy': 0.95875}, {'combination': ((0, 75), (150, 225)), 'accuracy': 0.938125}, {'combination': ((0, 75), (225, 300)), 'accuracy': 0.96875}, {'combination': ((0, 75), (300, 375)), 'accuracy': 0.924375}, {'combination': ((0, 75), (375, 450)), 'accuracy': 0.948125}, {'combination': ((0, 75), (450, 525)), 'accuracy': 0.926875}, {'combination': ((0, 75), (525, 600)), 'accuracy': 0.935}, {'combi

In [105]:
# Map the best combination indices to their corresponding text names
best_combination_text = ", ".join([name for name, indices in mapping.items() if indices in result["best_combination"]])
# print("Best Combination (Text):", best_combination_text)
display_highest_accuracy(result["best_accuracy"], best_combination_text)



<div style="border: 2px solid #424242; padding: 15px; border-radius: 12px; background-color: #212121; color: #E0E0E0;">
    <h2 style="color: #FFCA28; text-align: center; font-family: Arial, sans-serif;">🌟 Current Highest Accuracy 🌟</h2>
    <p style="font-size: 18px; text-align: center; font-family: Arial, sans-serif;">
        <strong>Accuracy:</strong> <span style="color: #76FF03; font-weight: bold;">0.971250</span><br>
        <strong>Combination:</strong> <span style="color: #29B6F6; font-weight: bold;">mfcc75mean, MFCC_min, MFCC_max</span>
    </p>
</div>


In [106]:
# Sort all accuracies in descending order
sorted_accuracies = sorted(result["all_accuracies"], key=lambda x: x["accuracy"], reverse=True)

# Write the sorted accuracies to a text file in tabular form with improved formatting
with open("statistics/sorted_accuracies_2000_MFCC_HFCC_extra_Stats_Gender_SVM.txt", "w") as f:
    # Write the header with borders
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")
    f.write(f"| {'Combination':<98} | {'Accuracy':<10} |\n")
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")
    
    # Write each combination and its accuracy with borders
    for entry in sorted_accuracies:
        combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
        f.write(f"| {combination_str:<98} | {entry['accuracy']:<10.6f} |\n")
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")

# Display the sorted combinations with their accuracies
for entry in sorted_accuracies:
    combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
    print(f"Combination: {combination_str}, Accuracy: {entry['accuracy']}")

Combination: mfcc75mean, MFCC_min, MFCC_max, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_min, MFCC_max, Hfcc75mean, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_min, MFCC_max, HFCC75median, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_min, MFCC_max, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, Hfcc75mean, HFCC_q25, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, HFCC75median, HFCC_q25, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, HFCC_q25, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_min, MFCC_max, Hfcc75mean, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, Hfcc75mean, HFCC75median, HFCC_q25, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, Hfcc75mean, HFCC_q25, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, HFCC75median, HFCC_q25, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75mean, MFCC_max, Hfcc75mean, HFCC75median, HFCC_q25, HFCC_q75, Accuracy: 0.97125
Combination: mfcc75std, MFCC_min, MFCC_max

In [107]:
from playsound import playsound

def play_sound(file_path):
    """
    Play a sound file.

    Args:
        file_path (str): Path to the sound file.
    """
    try:
        playsound(file_path)
        playsound(file_path)
    except Exception as e:
        print(f"Error playing sound: {e}")

play_sound("../sound/done.mp3")